# Getting Started - Vertex AI GenAI Python Client

Original source: [vertex-ai-mlops/Applied GenAI/Getting Started - Vertex AI GenAI Python Client.ipynb at main · statmike/vertex-ai-mlops](https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Getting%20Started%20-%20Vertex%20AI%20GenAI%20Python%20Client.ipynb)

How to get information into (prompts) and out of (responses) of Vertex AI hosted Generative AI Models!

Vertex AI has access to hosted [generative AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview) models, called foundation models. These can be easily interacted with in the console using [Vertex AI Generative AI Studio](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/generative-ai-studio). This notebook covers interacting with the models with the [`vertexai`](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai) Python package that is include with the Vertex AI SDK [aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest).

*   Gemini API (Multimodal data, text, code, chat)
    *   `vertexai.generative_models`
*   PaLM API (Text, chat, embeddings)
    *   `vertexai.language_models`
*   Codey APIs (Code generation, code chat, code completion)
    *   `vertexai.language_models`
*   Imagen API (Image generation, image editing, image captioning, visual question answering, and multimodal embedding)
    *   `vertexai.vision_models`

The current models in GA and in preview can be reviewed in the [documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models) as well as within the [Vertex AI Model Garden](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/model-garden) which also include access to serving and tuning OSS models.

Model have names and versions. The [names](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#model_naming_scheme) are designed show both the purpose and size of the model. For instance, sizes are conveyed with names like `Bison` and `Gecko` while purposes are `text`, `textembedding`, `chat`, `code`, `imagetext`, `imagegeneration` and more. [Versions](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/model-versioning) are either `stable` and available for six months after the release of subsequent version, or `latest` which are incremenatally updated and improved but can give different responses to the same prompt.


* * *


## Colab Setup

To run this notebook in Colab click <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/model_monitoring.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> and run the cells in this section. Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
# PROJECT_ID = 'your-project-id' # replace with project ID

In [21]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

In [22]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
print(PROJECT_ID)

demos-vertex-ai


## Install packages

In [1]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.storage', 'google-cloud-storage')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

In [2]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'demos-vertex-ai'

In [2]:
REGION = 'us-central1'
SERIES = 'generative-ai'
EXPERIMENT = 'getting-started'

# Set the BUCKET name for saving work:
BUCKET = PROJECT_ID

### Packages

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from google.cloud import aiplatform
from google.cloud import storage

import vertexai.vision_models # Imagen Models
import vertexai.preview.vision_models
import vertexai.language_models # PaLM and Codey Models
import vertexai.generative_models # for Gemini Models

import json
import requests
import IPython

In [4]:
aiplatform.__version__

'1.48.0'

### Clients

In [5]:
vertexai.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)

bucket = gcs.lookup_bucket(BUCKET)

* * *

## Vertex AI Package

With the [vertexai](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai) client there are packages for the types of data being interacted with. There is also a higher package for preview models (not yet in GA).

> **NOTE:** In can be helpful to review the API Documentation at it's source in GitHub for up to the moment release information: [github/googleapis/python-aiplatform](https://github.com/googleapis/python-aiplatform/tree/main)

Gemini Text and Multimodal Models:

*   [vertexai.generative\_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.generative_models)
    *   [vertexai.preview.generative\_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.generative_models)

Language Models (PaLM and Codey Models):

*   [vertexai.language\_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models)
    *   [vertexai.preview.language\_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.language_models)

Vision Models (Imagen Models):

*   [vertexai.vision\_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.vision_models)
    *   [vertexai.preview.vision\_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.vision_models)

In [6]:
# Gemini Models
gemini_text = vertexai.generative_models.GenerativeModel("gemini-1.0-pro")
gemini_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-vision")

# PaLM and Codey Models
text_model = vertexai.language_models.TextGenerationModel.from_pretrained('text-bison')
chat_model = vertexai.language_models.ChatModel.from_pretrained('chat-bison')
textembed_model = vertexai.language_models.TextEmbeddingModel.from_pretrained('textembedding-gecko')
codegen_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-bison')
codecomp_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-gecko')
codechat_model = vertexai.language_models.CodeChatModel.from_pretrained('codechat-bison')

# Imagen Models
imagecap_model = vertexai.vision_models.ImageCaptioningModel.from_pretrained("imagetext")
imageqna_model = vertexai.vision_models.ImageQnAModel.from_pretrained("imagetext")
imagetext_model = vertexai.vision_models.ImageTextModel.from_pretrained("imagetext")
multimodalembed_model = vertexai.vision_models.MultiModalEmbeddingModel.from_pretrained('multimodalembedding')
imagen1_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@002')
imagen2_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@005')

In [7]:
gemini_multimodal

In [15]:
from google.cloud import aiplatform
import vertexai

aiplatform.Model("/projects/demos-vertex-ai/locations/us-central1/publishers/google/models/gemini-1.0-pro-002")

ValueError: Resource /projects/demos-vertex-ai/locations/us-central1/publishers/google/models/gemini-1.0-pro-002 is not a valid resource ID.

* * *

## Gemini Models

[Reference: Gemini API](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini)

### Text Prompt

In [31]:
prompt = "What are the starting points for AI?"

In [32]:
response = gemini_text.generate_content(prompt)
response

candidates {
  content {
    role: "model"
    parts {
      text: "**Historical Foundations:**\n\n* **Ancient Greece:** Aristotle\'s syllogisms (logical reasoning)\n* **Medieval Europe:** Islamic scholars\' contributions to algebra and algorithms\n* **17th Century:** Ren\303\251 Descartes\' analytical geometry, Blaise Pascal\'s calculating machines\n* **19th Century:** Charles Babbage\'s mechanical computer (Analytical Engine)\n\n**Modern Era:**\n\n* **1950s:** Turing\'s test of AI intelligence\n* **1956:** Dartmouth Conference coined the term \"artificial intelligence\"\n* **1960s-1970s:** Winter of AI (funding and interest decline)\n* **1980s-1990s:** Expert systems, machine learning, neural networks\n* **2000s-Present:** Advancements in deep learning, cloud computing, big data\n\n**Theoretical underpinnings:**\n\n* **Cognitive Science:** Study of human cognition, providing models for AI development\n* **Computer Science:** Algorithmic principles, data structures, and computational 

In [33]:
IPython.display.Markdown(response.text)

**Historical Foundations:**

* **Ancient Greece:** Aristotle's syllogisms (logical reasoning)
* **Medieval Europe:** Islamic scholars' contributions to algebra and algorithms
* **17th Century:** René Descartes' analytical geometry, Blaise Pascal's calculating machines
* **19th Century:** Charles Babbage's mechanical computer (Analytical Engine)

**Modern Era:**

* **1950s:** Turing's test of AI intelligence
* **1956:** Dartmouth Conference coined the term "artificial intelligence"
* **1960s-1970s:** Winter of AI (funding and interest decline)
* **1980s-1990s:** Expert systems, machine learning, neural networks
* **2000s-Present:** Advancements in deep learning, cloud computing, big data

**Theoretical underpinnings:**

* **Cognitive Science:** Study of human cognition, providing models for AI development
* **Computer Science:** Algorithmic principles, data structures, and computational complexity
* **Cybernetics:** Study of feedback and control systems
* **Mathematics:** Linear algebra, statistics, probability theory, optimization

**Technological Advancements:**

* **Computation Power:** Increased computer speed and memory capacity
* **Big Data:** Availability of vast amounts of data for training AI models
* **Cloud Computing:** Provision of computational resources and storage on demand
* **Deep Learning:** Powerful neural networks inspired by human brain architecture
* **Natural Language Processing:** AI's ability to understand and generate human language

#### Streaming Response

In [16]:
for response in gemini_text.generate_content(prompt, stream = True):
    print(response.text)

**Key
 Starting Points for Artificial Intelligence (AI)**

**1. Data and Information:**
* Access to vast amounts of data for training and
 testing AI models.
* Techniques for gathering, cleaning, and preparing data that meets AI requirements.

**2. Algorithms and Models:**

* Mathematical and statistical models that define the core decision-making processes of AI.
* Training methods to optimize models based on available data and expected outcomes.

**3. Computing Power:**
* High-performance computing resources, such as graphics processing units (
GPUs), for efficient model training and inference.
* Access to cloud computing platforms that provide scalability and flexibility.

**4. Languages and Tools:**
* Specialized programming languages (e.g., Python, R) and software libraries for
 AI development.
* Frameworks and platforms for simplifying the development and deployment of AI solutions.

**5. Real-World Context:**
* Understanding the specific application domain and business goals for

#### Async Response

The client has built in method for awaitable responses that make it easy to make asynchronous request.

> For detailed coverage and examples of asynchronous call to these API's, scaling, error handling, and managing fail over regions check out this notebook in the same folder: [Python Asynchronous API Calls](https://github.com/statmike/vertex-ai-mlops/blob/e7092c381656dc119c5fb1a3dbb78523a4905dd2/Applied%20GenAI/Python%20Asynchronous%20API%20Calls.ipynb)

In [18]:
response = await gemini_text.generate_content_async(prompt)
IPython.display.Markdown(response.text)

**Origins and Early Developments:**

* **1950s:**
    * Dartmouth Summer Research Project on Artificial Intelligence (1956)
    * Perceptrons (1957) by Frank Rosenblatt
* **1960s:**
    * Natural language processing (NLP) and expert systems
    * Development of artificial neural networks (ANNs)

**Modern Developments:**

* **1990s:**
    * Introduction of deep neural networks (DNNs)
    * Rise of machine learning (ML) and data mining
* **2000s:**
    * Cloud computing and distributed computing
    * Big Data and data analytics
    * Emergence of autonomous systems and robotics

**Specific Fields and Applications:**

* **Computer vision:** Image and video recognition
* **Natural language processing:** Language understanding and generation
* **Machine learning:** Algorithms that learn from data
* **Robotics:** Autonomous movement and control
* **Expert systems:** Computer programs that simulate human expertise
* **Data mining:** Extraction of knowledge from large datasets

**Key Technology Advancements:**

* **Hardware advancements:** GPUs, TPUs, and specialized AI chips
* **Algorithm improvements:** Deep learning, reinforcement learning, and transfer learning
* **Data availability:** Large datasets and open-source repositories
* **Computational resources:** Cloud computing and distributed systems

#### Parameters For Configuration

[Reference: Gemini API](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini)

In [19]:
response = gemini_text.generate_content(
    prompt,
    generation_config = vertexai.generative_models.GenerationConfig(
        temperature = 0.5,
        top_p = .8,
        top_k = 25,
        candidate_count = 1, 
        max_output_tokens = 500
    )
)
response

candidates {
  content {
    role: "model"
    parts {
      text: "**Historical Starting Points:**\n\n* **1943:** Warren McCulloch and Walter Pitts develop the first artificial neuron model.\n* **1950:** Alan Turing proposes the Turing Test as a measure of machine intelligence.\n* **1956:** The Dartmouth Conference establishes the field of artificial intelligence.\n\n**Theoretical Foundations:**\n\n* **Logic and Reasoning:** Formalizing logical rules and reasoning processes to enable machines to make inferences.\n* **Machine Learning:** Algorithms that allow machines to learn from data without explicit programming.\n* **Search and Optimization:** Techniques for finding optimal solutions to complex problems.\n* **Natural Language Processing:** Understanding and generating human language.\n* **Computer Vision:** Processing visual data to extract meaningful information.\n\n**Technological Advancements:**\n\n* **1970s-1980s:** Development of expert systems and rule-based AI.\n* **1990s:**

In [20]:
IPython.display.Markdown(response.text)

**Historical Starting Points:**

* **1943:** Warren McCulloch and Walter Pitts develop the first artificial neuron model.
* **1950:** Alan Turing proposes the Turing Test as a measure of machine intelligence.
* **1956:** The Dartmouth Conference establishes the field of artificial intelligence.

**Theoretical Foundations:**

* **Logic and Reasoning:** Formalizing logical rules and reasoning processes to enable machines to make inferences.
* **Machine Learning:** Algorithms that allow machines to learn from data without explicit programming.
* **Search and Optimization:** Techniques for finding optimal solutions to complex problems.
* **Natural Language Processing:** Understanding and generating human language.
* **Computer Vision:** Processing visual data to extract meaningful information.

**Technological Advancements:**

* **1970s-1980s:** Development of expert systems and rule-based AI.
* **1990s:** Emergence of neural networks and deep learning.
* **2000s:** Advancements in computing power and data availability.
* **2010s:** Widespread adoption of AI in various industries.

**Current Trends:**

* **Deep Learning:** Neural networks with multiple hidden layers that can learn complex patterns from data.
* **Big Data and Cloud Computing:** Access to vast amounts of data and computational resources.
* **Edge Computing:** AI algorithms deployed on devices with limited resources.
* **Generative AI:** Creating new data or content based on existing patterns.
* **Ethical Considerations:** Addressing concerns about bias, privacy, and the potential impact of AI on society.

#### Grounding With Vertex AI Search & Conversation

**in preview, need to load preview model client**

Automatically ground prompts in the API call by providing a link to a retrieval source. In this case Vertex AI Search & Conversation data stores can be directly referenced for context retrieval.

This section uses a data store created in this repository: [Vertex AI Search](https://github.com/statmike/vertex-ai-mlops/blob/e7092c381656dc119c5fb1a3dbb78523a4905dd2/Applied%20GenAI/Vertex%20AI%20Search/readme.md)

*   A data store and search application are created using the official rules manual of baseball

Reference: [Grounding Model Responses](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini)

In [ ]:
#gemini_text_preview = vertexai.preview.generative_models.GenerativeModel("gemini-pro")#("gemini-1.0-pro")
# from vertexai.preview.generative_models import GenerationConfig, grounding, Tool

In [ ]:
# prompt_baseball = "Is it a homerun when the ball is deflected by a fielder over the outfield fence? Explain."

### PaLM

In [36]:
response = text_model.predict(prompt)
response.text

' The starting points for AI can be traced back to several key moments and developments in the field of computer science and technology:\n\n1. **Early Theoretical Foundations**:\n   - **1940s-1950s**: The birth of AI can be linked to the work of pioneers like Alan Turing, John von Neumann, and Norbert Wiener. Turing\'s concept of a "universal computing machine" and his "Turing test" laid the groundwork for the idea of intelligent machines.\n\n2. **Dartmouth Summer Research Project (1956)**:\n   - This workshop, often considered the birthplace of AI'